In [2]:
# Importing libraries
import pandas as pd 
import numpy as np 
from pathlib import Path 

pd.options.display.max_columns = None
pd.options.display.width = 120

In [3]:
# Loading the dataset
file_path = Path(r"C:\Users\ishim\urban_mobility\data\yellow_tripdata_2025-08.parquet")

In [6]:
# Reading the dataset
data = pd.read_parquet(file_path)

In [8]:
# Printing the dimensions of my dataset
print("Original shape", data.shape)

Original shape (3574091, 20)


In [9]:
# Data inspection
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3574091 entries, 0 to 3574090
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [11]:
# Selecting columns  with numbers ?(floats and integers)
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns

In [13]:
# Replacing the missing values with 0 to prevent errors when calculating
data[numeric_columns] = data[numeric_columns].fillna(0)

In [18]:
# Verifying if date and times are stored in the correct datetime format
for current_column in ['tpep_pickup_datetime', 'tpep_dropoff_datetime']:
    data[current_column] = pd.to_datetime(data[current_column], errors='coerce')

In [19]:
# Calculating the trip duration
trip_duration = data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime'] 

In [23]:
# Converting trip duration into minutes and converting negative or null durations to 0
data['trip_duration_min'] = (
    (data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime']).dt.total_seconds() / 60).fillna(0)

In [25]:
# Define all fare components
fare_components = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'Airport_fee']

# Check if all columns exist and replace them with 0 if not
for column in fare_components:
    if column not in data.columns:
        data[column] = 0


In [29]:
# Manually calculate the total fare amount for all components
data['calculated_total_amount'] = data[fare_components].sum(axis=1)

In [ ]:
# Compare the calculated total with the generated total_amount column
tolerance = 5 # It will allow a difference of 5 cents
data['difference'] = abs(data['total_amount'] - data['calculated_total_amount'])

In [34]:
# Rows where the difference is greater than the tolerance
mismatched_totals = data['difference'] > tolerance

In [ ]:
# 
if 'payment_type' in data.columns:
    cash_mask = data['payment_type'] == 2



In [38]:
clean_data = data[
    (data['trip_distance'] > 0) &
    (data['trip_duration_min'] > 0)
].copy()
print("Cleaned shape:", clean_data.shape)


Cleaned shape: (3422739, 23)


In [39]:
clean_data = clean_data[
    (clean_data['trip_duration_min'] < 240) &  # < 4 hours
    (clean_data['trip_distance'] < 100)        # < 100 miles
]


In [41]:
clean_data['speed_mph'] = clean_data['trip_distance'] / (clean_data['trip_duration_min'] / 60)
print("Final cleaned shape:", clean_data.shape)

Final cleaned shape: (3421428, 24)


In [43]:
# --- 12. Additional cleaning filters ---

# Remove trips with zero or extremely high passenger counts
clean_data = clean_data[
    (clean_data['passenger_count'] > 0) &
    (clean_data['passenger_count'] <= 6)  # realistic upper bound
]

# Remove invalid or unknown payment types
valid_payment_types = [1, 2, 3, 4, 5, 6]  # standard NYC Taxi codes
clean_data = clean_data[clean_data['payment_type'].isin(valid_payment_types)]

# Drop rows with missing pickup or dropoff locations
clean_data = clean_data.dropna(subset=['PULocationID', 'DOLocationID'])

# Remove outliers in fare or tip amounts (e.g., extreme high values)
clean_data = clean_data[
    (clean_data['fare_amount'].between(1, 500)) &
    (clean_data['tip_amount'].between(0, 200))
]

# Optionally remove forwarded trips if not needed
clean_data = clean_data[clean_data['store_and_fwd_flag'] != 'Y']

print("Final cleaned shape:", clean_data.shape)


Final cleaned shape: (2506343, 24)
